In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../") 

import os
import yaml
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42
import matplotlib.gridspec as gridspec
plt.rcParams["font.family"] = "Optima"
plt.rcParams["font.weight"] = "light"

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import arviz as az

from epimodel import preprocess_data, run_model, EpidemiologicalParameters
from epimodel.plotting.intervention_plots import combine_npi_samples

In [ ]:
default_res = az.from_netcdf('final_results2.netcdf')
default_alpha = default_res.posterior.alpha_i.data.reshape((5000, 19))

data = preprocess_data('../../data/all_merged_data_2021-01-22.csv')
data.featurize() 
data.mask_new_variant(new_variant_fraction_fname='../../data/nuts3_new_variant_fraction.csv')
data.mask_from_date('2021-01-09') 

default_names = data.CMs

In [ ]:
corrected_names = [
    'Night clubs closed',
    'Gastronomy closed',
    'Leisure venues closed',
    'Retail and close-contact\nservices closed',
    'Night time curfew',
    'Primary schools closed',
    'Secondary schools closed',
    'Universities closed',
    'All public gatherings banned',
    'Public gatherings limited to 2 people', 
    'Public gatherings limited to ≤10 people',
    'Public gatherings limited to ≤30 people', 
    'Public gatherings limited to 2 households',
    'All household mixing in private banned',
    'Household mixing in private limited to 2 people', 
    'Household mixing in private limited to ≤10 people',
    'Household mixing in private limited to ≤30 people', 
    'Household mixing in private limited to 2 households',
    'Stricter mask-wearing\npolicy',
]
grouped_npis = {
    'All non-essential\nbusinesses closed': {
        'npis': ['Retail Closed', 'Some Face-to-Face Businesses Closed', 
                'Gastronomy Closed', 'Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
        'main': True,
    },
    'Night clubs closed': {
        'npis': ['Some Face-to-Face Businesses Closed'],
        'type': "exclude",
        'color': cols[0],
        'main': False,
    },
    'Leisure and entertainment\nvenues closed': {
        'npis': ['Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Gastronomy closed': {
        'npis': ['Gastronomy Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Retail and close-contact\nservices closed': {
        'npis': ['Retail Closed'],
        'type': 'exclude',
        'color': cols[0],
        'main': False,
    },
    'All gatherings banned': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': True
    },
    'All gatherings limited to 2 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤10 people\nfrom 2 households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'All educational\ninstitutions closed': {
        'npis': ['Primary Schools Closed', 'Secondary Schools Closed', 'Universities Away'],
        'type': 'exclude',
        'color': cols[2],
        'main': True
    },
    'Night time curfew': {
        'npis': ['Curfew'],
        'type': 'exclude',
        'color': cols[1],
        'main': True
    },
    'Stricter mask-wearing\npolicy': {
        'npis': ['Mandatory Mask Wearing >= 3'],
        'type': 'exclude',
        'color': cols[1],
        'main': True
    },
    'All public gatherings banned': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to 2 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        "type": "exclude",
        'color': cols[4],
    },
    'Public gatherings limited to ≤10 people\nfrom 2 households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'All household mixing in private banned': {
        'npis': ['Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to 2 people': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤10 people from 2 households': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Combined effect of all NPIs': {
        'npis': ['Some Face-to-Face Businesses Closed',
                 'Gastronomy Closed',
                 'Leisure Venues Closed',
                 'Retail Closed',
                 'Curfew',
                 'Primary Schools Closed',
                 'Secondary Schools Closed',
                 'Universities Away',
                 'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit',
                 'Mandatory Mask Wearing >= 3'],
        "type": 'include'
    }
}

grouped_npis_weaker = {
    'All non-essential\nbusinesses closed': {
        'npis': ['Retail Closed', 'Some Face-to-Face Businesses Closed', 
                'Gastronomy Closed', 'Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
        'main': True,
    },
    'Night clubs closed': {
        'npis': ['Some Face-to-Face Businesses Closed'],
        'type': "exclude",
        'color': cols[0],
        'main': False,
    },
    'Leisure and entertainment\nvenues closed': {
        'npis': ['Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Gastronomy closed': {
        'npis': ['Gastronomy Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Retail and close-contact\nservices closed': {
        'npis': ['Retail Closed'],
        'type': 'exclude',
        'color': cols[0],
        'main': False,
    },
    'All educational\ninstitutions closed': {
        'npis': ['Primary Schools Closed', 'Secondary Schools Closed', 'Universities Away'],
        'type': 'exclude',
        'color': cols[2],
        'main': True
    },
    'Schools closed': {
        'npis': ['Primary Schools Closed', 'Secondary Schools Closed'],
        'type': 'exclude',
        'color': cols[2],
        'main': False,
    },
    'Universities closed': {
        'npis': ['Universities Away'],
        'type': 'exclude',
        'color': cols[2],
        'main': False,
    },
    'All gatherings banned': {
        'npis': ['Public Gathering Person Limit - 1',
                 'Public Gathering Person Limit - 2',
                 'Public Gathering Person Limit - 10',
                 'Public Gathering Person Limit - 30',
                 'Extra Public Household Limit',
                 'Private Gathering Person Limit - 1',
                 'Private Gathering Person Limit - 2',
                 'Private Gathering Person Limit - 10',
                 'Private Gathering Person Limit - 30',
                 'Extra Private Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': True
    },
    'All gatherings limited to 2 people': {
        'npis': [#'Public Gathering Person Limit - 1',
                 'Public Gathering Person Limit - 2',
                 'Public Gathering Person Limit - 10',
                 'Public Gathering Person Limit - 30',
                 'Extra Public Household Limit',
                 #'Private Gathering Person Limit - 1',
                 'Private Gathering Person Limit - 2',
                 'Private Gathering Person Limit - 10',
                 'Private Gathering Person Limit - 30',
                 'Extra Private Household Limit'
                ],
        "type": "exclude",
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤10 people\nfrom 2 households': {
        'npis': [#'Public Gathering Person Limit - 1',
                 #'Public Gathering Person Limit - 2',
                 'Public Gathering Person Limit - 10',
                 'Public Gathering Person Limit - 30',
                 'Extra Public Household Limit',
                 #'Private Gathering Person Limit - 1',
                 #'Private Gathering Person Limit - 2',
                 'Private Gathering Person Limit - 10',
                 'Private Gathering Person Limit - 30',
                 'Extra Private Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤10 people': {
        'npis': [#'Public Gathering Person Limit - 1',
                 #'Public Gathering Person Limit - 2',
                 'Public Gathering Person Limit - 10',
                 'Public Gathering Person Limit - 30',
                 #'Extra Public Household Limit',
                 #'Private Gathering Person Limit - 1',
                 #'Private Gathering Person Limit - 2',
                 'Private Gathering Person Limit - 10',
                 'Private Gathering Person Limit - 30',
                 #'Extra Private Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤30 people': {
        'npis': [#'Public Gathering Person Limit - 1',
                 #'Public Gathering Person Limit - 2',
                 #'Public Gathering Person Limit - 10',
                 'Public Gathering Person Limit - 30',
                 #'Extra Public Household Limit',
                 #'Private Gathering Person Limit - 1',
                 #'Private Gathering Person Limit - 2',
                 #'Private Gathering Person Limit - 10',
                 'Private Gathering Person Limit - 30',
                 #'Extra Private Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'Night time curfew': {
        'npis': ['Curfew'],
        'type': 'exclude',
        'color': cols[1],
        'main': True
    },
    'Stricter mask-wearing\npolicy': {
        'npis': ['Mandatory Mask Wearing >= 3'],
        'type': 'exclude',
        'color': cols[1],
        'main': True
    },
    'All public gatherings banned': {
        'npis': ['Public Gathering Person Limit - 1',
                 'Public Gathering Person Limit - 2',
                 'Public Gathering Person Limit - 10',
                 'Public Gathering Person Limit - 30',
                 'Extra Public Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to 2 people': {
        'npis': [#'Public Gathering Person Limit - 1',
                 'Public Gathering Person Limit - 2',
                 'Public Gathering Person Limit - 10',
                 'Public Gathering Person Limit - 30',
                 'Extra Public Household Limit',
                ],
        "type": "exclude",
        'color': cols[4],
    },
    'Public gatherings limited to ≤10 people\nfrom 2 households': {
        'npis': [#'Public Gathering Person Limit - 1',
                 #'Public Gathering Person Limit - 2',
                 'Public Gathering Person Limit - 10',
                 'Public Gathering Person Limit - 30',
                 'Extra Public Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤10 people': {
        'npis': [#'Public Gathering Person Limit - 1',
                 #'Public Gathering Person Limit - 2',
                 'Public Gathering Person Limit - 10',
                 'Public Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤30 people': {
        'npis': [#'Public Gathering Person Limit - 1',
                 #'Public Gathering Person Limit - 2',
                 #'Public Gathering Person Limit - 10',
                 'Public Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'All household mixing in private banned': {
        'npis': ['Private Gathering Person Limit - 1',
                 'Private Gathering Person Limit - 2',
                 'Private Gathering Person Limit - 10',
                 'Private Gathering Person Limit - 30',
                 'Extra Private Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to 2 people': {
        'npis': [#'Private Gathering Person Limit - 1',
                 'Private Gathering Person Limit - 2',
                 'Private Gathering Person Limit - 10',
                 'Private Gathering Person Limit - 30',
                 'Extra Private Household Limit'
                ],
        "type": "exclude",
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤10 people from 2 households': {
        'npis': [#'Private Gathering Person Limit - 1',
                 #'Private Gathering Person Limit - 2',
                 'Private Gathering Person Limit - 10',
                 'Private Gathering Person Limit - 30',
                 'Extra Private Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤10 people': {
        'npis': [#'Public Gathering Person Limit - 1',
                 #'Extra Public Household Limit',
                 #'Private Gathering Person Limit - 1',
                 #'Private Gathering Person Limit - 2',
                 'Private Gathering Person Limit - 10',
                 'Private Gathering Person Limit - 30',
                 #'Extra Private Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤30 people': {
        'npis': [#'Public Gathering Person Limit - 1',
                 #'Public Gathering Person Limit - 2',
                 #'Public Gathering Person Limit - 10',
                 #'Extra Public Household Limit',
                 #'Private Gathering Person Limit - 1',
                 #'Private Gathering Person Limit - 2',
                 #'Private Gathering Person Limit - 10',
                 'Private Gathering Person Limit - 30',
                 #'Extra Private Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Combined effect of all NPIs': {
        'npis': ['Some Face-to-Face Businesses Closed',
                 'Gastronomy Closed',
                 'Leisure Venues Closed',
                 'Retail Closed',
                 'Curfew',
                 'Primary Schools Closed',
                 'Secondary Schools Closed',
                 'Universities Away',
                 'Public Gathering Person Limit - 1',
                 'Public Gathering Person Limit - 2',
                 'Public Gathering Person Limit - 10',
                 'Public Gathering Person Limit - 30',
                 'Extra Public Household Limit',
                 'Private Gathering Person Limit - 1',
                 'Private Gathering Person Limit - 2',
                 'Private Gathering Person Limit - 10',
                 'Private Gathering Person Limit - 30',
                 'Extra Private Household Limit',
                 'Mandatory Mask Wearing >= 3'],
        "type": 'include'
    }
}

grouped_npis_none = {
    'All non-essential\nbusinesses closed': {
        'npis': ['Retail Closed', 'Some Face-to-Face Businesses Closed', 
                'Gastronomy Closed', 'Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
        'main': True,
    },
    'Night clubs closed': {
        'npis': ['Some Face-to-Face Businesses Closed'],
        'type': "exclude",
        'color': cols[0],
        'main': False,
    },
    'Leisure and entertainment\nvenues closed': {
        'npis': ['Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Gastronomy closed': {
        'npis': ['Gastronomy Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Retail and close-contact\nservices closed': {
        'npis': ['Retail Closed'],
        'type': 'exclude',
        'color': cols[0],
        'main': False,
    },
    'All educational\ninstitutions closed': {
        'npis': ['Primary Schools Closed', 'Secondary Schools Closed', 'Universities Away'],
        'type': 'exclude',
        'color': cols[2],
        'main': True
    },
    'Schools closed': {
        'npis': ['Primary Schools Closed', 'Secondary Schools Closed'],
        'type': 'exclude',
        'color': cols[2],
        'main': False,
    },
    'Universities closed': {
        'npis': ['Universities Away'],
        'type': 'exclude',
        'color': cols[2],
        'main': False,
    },
    'All gatherings banned': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit',
                 'Public Outdoor Gathering Person Limit - 1',
                 'Public Outdoor Gathering Person Limit - 2',
                 'Public Outdoor Gathering Person Limit - 10',
                 'Public Outdoor Gathering Person Limit - 30',
                 'Extra Public Outdoor Household Limit',
                 'Private Outdoor Gathering Person Limit - 1',
                 'Private Outdoor Gathering Person Limit - 2',
                 'Private Outdoor Gathering Person Limit - 10',
                 'Private Outdoor Gathering Person Limit - 30',
                 'Extra Private Outdoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': True
    },
    'All gatherings limited to 2 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit',
                 #'Public Indoor Gathering Person Limit - 1',
                 'Public Outdoor Gathering Person Limit - 2',
                 'Public Outdoor Gathering Person Limit - 10',
                 'Public Outdoor Gathering Person Limit - 30',
                 'Extra Public Outdoor Household Limit',
                 #'Private Outdoor Gathering Person Limit - 1',
                 'Private Outdoor Gathering Person Limit - 2',
                 'Private Outdoor Gathering Person Limit - 10',
                 'Private Outdoor Gathering Person Limit - 30',
                 'Extra Private Outdoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤10 people\nfrom 2 households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit',
                 #'Public Outdoor Gathering Person Limit - 1',
                 #'Public Outdoor Gathering Person Limit - 2',
                 'Public Outdoor Gathering Person Limit - 10',
                 'Public Outdoor Gathering Person Limit - 30',
                 'Extra Public Outdoor Household Limit',
                 #'Private Outdoor Gathering Person Limit - 1',
                 #'Private Outdoor Gathering Person Limit - 2',
                 'Private Outdoor Gathering Person Limit - 10',
                 'Private Outdoor Gathering Person Limit - 30',
                 'Extra Private Outdoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit',
                #'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Outdoor Gathering Person Limit - 10',
                 'Public Outdoor Gathering Person Limit - 30',
                 #'Extra Public Outdoor Household Limit',
                 #'Private Outdoor Gathering Person Limit - 1',
                 #'Private Outdoor Gathering Person Limit - 2',
                 'Private Outdoor Gathering Person Limit - 10',
                 'Private Outdoor Gathering Person Limit - 30',
                 #'Extra Private Outdoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit',
             'Public Outdoor Gathering Person Limit - 30',
                 #'Extra Public Outdoor Household Limit',
                 #'Private Outdoor Gathering Person Limit - 1',
                 #'Private Outdoor Gathering Person Limit - 2',
                 #'Private Outdoor Gathering Person Limit - 10',
                 'Private Outdoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'Night time curfew': {
        'npis': ['Curfew'],
        'type': 'exclude',
        'color': cols[1],
        'main': True
    },
    'Stricter mask-wearing\npolicy': {
        'npis': ['Mandatory Mask Wearing >= 3'],
        'type': 'exclude',
        'color': cols[1],
        'main': True
    },
    'All public gatherings banned': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 'Public Outdoor Gathering Person Limit - 1',
                 'Public Outdoor Gathering Person Limit - 2',
                 'Public Outdoor Gathering Person Limit - 10',
                 'Public Outdoor Gathering Person Limit - 30',
                 'Extra Public Outdoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to 2 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 'Public Outdoor Gathering Person Limit - 2',
                 'Public Outdoor Gathering Person Limit - 10',
                 'Public Outdoor Gathering Person Limit - 30',
                 'Extra Public Outdoor Household Limit',
                ],
        "type": "exclude",
        'color': cols[4],
    },
    'Public gatherings limited to ≤10 people\nfrom 2 households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 'Public Outdoor Gathering Person Limit - 10',
                 'Public Outdoor Gathering Person Limit - 30',
                 'Extra Public Outdoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Public Outdoor Gathering Person Limit - 10',
                 'Public Outdoor Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Public Outdoor Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'All household mixing in private banned': {
        'npis': ['Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit',
                 'Private Outdoor Gathering Person Limit - 1',
                 'Private Outdoor Gathering Person Limit - 2',
                 'Private Outdoor Gathering Person Limit - 10',
                 'Private Outdoor Gathering Person Limit - 30',
                 'Extra Private Outdoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to 2 people': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit',
                 'Private Outdoor Gathering Person Limit - 2',
                 'Private Outdoor Gathering Person Limit - 10',
                 'Private Outdoor Gathering Person Limit - 30',
                 'Extra Private Outdoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤10 people from 2 households': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit',
                 'Private Outdoor Gathering Person Limit - 10',
                 'Private Outdoor Gathering Person Limit - 30',
                 'Extra Private Outdoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Outdoor Gathering Person Limit - 10',
                 'Private Outdoor Gathering Person Limit - 30',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Private Outdoor Gathering Person Limit - 30'
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Combined effect of all NPIs': {
        'npis': ['Some Face-to-Face Businesses Closed',
                 'Gastronomy Closed',
                 'Leisure Venues Closed',
                 'Retail Closed',
                 'Curfew',
                 'Primary Schools Closed',
                 'Secondary Schools Closed',
                 'Universities Away',
                 'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Public Outdoor Gathering Person Limit - 1',
                 'Public Outdoor Gathering Person Limit - 2',
                 'Public Outdoor Gathering Person Limit - 10',
                 'Public Outdoor Gathering Person Limit - 30',
                 'Extra Public Outdoor Household Limit',
                 'Private Outdoor Gathering Person Limit - 1',
                 'Private Outdoor Gathering Person Limit - 2',
                 'Private Outdoor Gathering Person Limit - 10',
                 'Private Outdoor Gathering Person Limit - 30',
                 'Extra Private Outdoor Household Limit',
                 'Mandatory Mask Wearing >= 3'],
        "type": 'include'
    }
}



In [ ]:
cols = sns.color_palette('colorblind')

In [ ]:
import textwrap
import json

def intervention_prior_labeler(d): 
    if float(d['exp_config']['intervention_prior']['scale']) == 20.0:
        return  "AsymmetricLaplace(0, 0.5, 20)"
    elif d['exp_config']['intervention_prior']['type'] == "normal":
        return f"Normal(0, {d['exp_config']['intervention_prior']['scale']}$^2$)"
    elif float(d['exp_config']['intervention_prior']['scale']) == 0.15:
        return "HalfNormal(0, 0.15$^2$)"

def get_all_experiments(path):
    experiments = []
    for subdir, dirs, files in os.walk(f'{path}'):
        for filename in files:
#             if filename.endswith('.yaml'):
#                 filepath = subdir + os.sep + filename
#                 print(filepath)
#                 with open(filepath) as f:
#                     try:
#                         data = yaml.safe_load(f)
#                         experiments.append(data)
#                     except:
#                         print('failed to load f')
            if filename.endswith('.json'):
                filepath = subdir + os.sep + filename
                print(filepath)
                with open(filepath) as f:
                    try:
                        data = json.load(f)
                        experiments.append(data)
                    except:
                        print('failed to load f')
    return experiments

def filter_by_exp_tag(experiments, exp_tag):
    filtered = []
    for experiment in experiments:
        if experiment['exp_tag'] == exp_tag:
                    filtered.append(experiment)
    return filtered


all_exp_info = {
    "basic_R_prior_mean": {
        "title": "$\\tilde{R}_{0, l}$ Prior: Mean",
        "labeler": lambda d: d['exp_config']["basic_R_prior"]["mean"],
        "default_label": "1.35"
    },
    "basic_R_prior_scale": {
        "title": "$\\tilde{R}_{0, l}$ Prior: Scale",
        "labeler": lambda d: d['exp_config']["basic_R_prior"]["variability"],
        "default_label": "0.3"
    },
    "cases_delay_mean": {
        "title": "Mean Delay from Infection to Case Reporting",
        "labeler": lambda d: f"{float(d['exp_config']['cases_delay_mean_shift']):+.2f} days",
        "default_label": "Default"
    },
    "death_delay_mean": {
        "title": "Mean Delay from Infection to Death Reporting",
        "labeler": lambda d: f"{float(d['exp_config']['deaths_delay_mean_shift']):+.2f} days",
        "default_label": "Default"
    },
    "gen_int_mean": {
        "title": "Generation Interval Mean",
        "labeler": lambda d: f"{float(d['exp_config']['gen_int_mean_shift']):+.2f} days",
        "default_label": "Default"
    },
    "infection_noise_scale": {
        "title": "Infection Noise Scale",
        "labeler": lambda d: f"{float(d['exp_config']['infection_noise_scale']):+.1f}",
        "default_label": "Default (5.0)"
    },
    "maximum_fraction_voc": {
        "title": "Maximum Percentage Infected\n with the Variant of Concern",
        "labeler": lambda d: f"{100*float(d['exp_config']['maximum_fraction_voc']):.0f}%",
        "default_label": "Default (10%)"
    },
    "output_noise_scale_prior": {
        "title": "Output Noise Scale Prior",
        "labeler": lambda d: f"HalfNormal({float(d['exp_config']['output_noise_scale_prior']):.2f})",
        "default_label": "Default (HalfNormal(5))"
    },
    "r_walk_noise_scale_prior": {
        "title": "Random Walk Noise Scale Prior",
        "labeler": lambda d: f"HalfNormal({float(d['exp_config']['r_walk_noise_scale_prior']):.2f})",
        "default_label": "Default (HalfNormal(0.15))"
    },
    "seeding_scaling": {
        "title": "Seeded Infection Prior",
        "labeler": lambda d: f"Lognormal(0, {float(d['exp_config']['seeding_scale']):.2f})",
        "default_label": "Default (Lognormal(0,3))"
    },
    "npi_leaveout": {
        "title": "NPI Leaveout",
        "labeler": lambda d: f"{textwrap.fill(corrected_names[d['exp_config']['npis'][0]], 15)}",
        "default_label": "Default"
    },
    "r_walk_period": {
        "title": "Random Walk Period",
        "labeler": lambda d: f"{d['exp_config']['r_walk_period']} days",
        "default_label": "Default (7 days)"
    },
    "intervention_prior": {
        "title": "Intervention Effect Prior",
        "labeler": intervention_prior_labeler,
        "default_label": "AsymmetricLaplace(0, 0.5, 30)"
    },
    "leaveout": {
        "title": "Country Leaveout",
        "labeler": lambda d: d['exp_config']['country_names'][0],
        "default_label": "Default"
    },
    "seeding_days": {
        "title": "Infection Seeding Days",
        "labeler": lambda d: f"{d['exp_config']['n_days_seeding']} days",
        "default_label": "7 days"
    },
    "boostrap": {
        "title": "Bootstrap",
        "labeler": lambda d: "Bootstrapped",
        "default_label": "Default"
    },
    "gatherings_agg": {
        "title": "Gatherings Aggregation",
        "labeler": lambda d: "Keep Both" if d['exp_config']['gatherings_aggregation'] == "none" else "Weaker",
        "default_label": "Default (Drop Outdoors)"
    },
    "delay_schools": {
        "title": "Schools Delayed",
        "labeler": lambda d: f"Schools Delayed\n({d['exp_config']['n_days_delay']} days)",
        "default_label": "Default"
    },
    "window_of_analysis": {
        "title": "Window of Analysis",
        "labeler": lambda d: f"{d['exp_config']['start_date']}\nto {d['exp_config']['end_date']}",
        "default_label": "Default"
    },
    "eng_ifr_iar": {
        "title": "Undercounting Adjustment",
        "labeler": lambda d: f"Time-Varying\nIFR/IAR Adjustment",
        "default_label": "Default"
    }
}

class experiment_type():
    def __init__(self, experiments, exp_info, tag):
        self.exp_info = exp_info
        self.experiments = experiments
        self.exp_info["tag"] = tag
        
        if "alpha_i" in list(self.experiments[0].keys()):
            self.experiments.sort(key=lambda x: np.median(np.array(x['alpha_i'])[:, 0]))

def get_unique_exp_tags(experiments):
    return list(np.unique([exp['exp_tag'] for exp in experiments]))

def make_all_experiment_classes(all_experiments):
    tags = get_unique_exp_tags(all_experiments)
    classes = []
    for tag in tags:
        if tag == "default":
            continue
        filtered_exps = filter_by_exp_tag(all_experiments, tag)
        if tag in all_exp_info.keys():
            exp_info = all_exp_info[tag]
            classes.append(experiment_type(filtered_exps, exp_info, tag))
        else:
            print(f"{tag} was skipped")
        
    return classes

In [ ]:
def add_trace_to_plot(samples, y_off, col, label, alpha, width, npi_comb_dict, cm_names, size=6):
    comb_effects, new_names = combine_npi_samples(npi_comb_dict, samples, cm_names)    
    comb_effects = 100*(1-np.exp(-comb_effects))
    npi_order = list(npi_comb_dict.keys())
    nF = len(npi_order)

    y_vals = -np.array([npi_order.index(name) for name in new_names])
    plt.plot([100], [100], color=col, linewidth=1, alpha=alpha, label=label)

    li, lq, m, uq, ui = np.percentile(comb_effects, [2.5, 25, 50, 75, 97.5], axis=0)
    plt.scatter(m, y_vals+y_off, marker="o", color=col, s=size, alpha=alpha, facecolor='white', zorder=3, linewidth=width/2)
    for cm in range(len(new_names)):
        plt.plot([li[cm], ui[cm]], [y_vals[cm]+y_off, y_vals[cm]+y_off], color=col, alpha=alpha*0.25, linewidth=width, zorder=2)
        plt.plot([lq[cm], uq[cm]], [y_vals[cm]+y_off, y_vals[cm]+y_off], color=col, alpha=alpha*0.75, linewidth=width, zorder=2)


def setup_plot(experiment_class, npi_comb_dict, y_ticks = True, xlabel=True, x_lims=(-25, 50), newfig=True):
    if newfig:
        plt.figure(figsize=(4, 6), dpi=400)
        
    ax = plt.gca()
    x_min, x_max = x_lims
    
    npi_order = list(npi_comb_dict.keys())
    plt.plot([0, 0], [1, -(len(npi_order)+2)], "--k", linewidth=0.5)

    xrange = np.array([x_min, x_max])
    
    for height in range(0, len(npi_order)+2, 2):
        plt.fill_between(xrange, -(height-0.5), -(height+0.5), color="silver", alpha=0.25, linewidth=0)
    xtick_vals = [-25, 0, 25, 50, 75, 100]
    xtick_str = [f"{x:.0f}%" for x in xtick_vals]
    
    if y_ticks:
        plt.yticks(-np.arange(len(npi_order)), npi_order, fontsize=6)
    else:
        plt.yticks([])
   
    plt.xticks(xtick_vals, xtick_str, fontsize=8)
    plt.xlim([x_min, x_max])
    plt.ylim([-(len(npi_order) - 0.25), 0.75])
    
    plt.plot([-100, 100], [-len(npi_order)+11.5, -len(npi_order)+11.5], 'k')

    if xlabel:
        plt.xlabel("Reduction in R", fontsize=8)

colors = [*sns.color_palette("colorblind"), *sns.color_palette("dark")]

def plot_experiment_class(experiment_class, npi_comb_dict, x_lims=None, default_res=None, default_names=None, width=1, newfig=True):
    default_label = experiment_class.exp_info["default_label"]
    labeler = experiment_class.exp_info["labeler"]
    title = experiment_class.exp_info["title"]
    
    n_max = 6
    
    if len(experiment_class.experiments) > n_max:
        n_plots = int(np.ceil(len(experiment_class.experiments) / n_max))
        all_indices = np.array_split(range(len(experiment_class.experiments)), n_plots)
        for j, indices in enumerate(all_indices):
            setup_plot(experiment_class, npi_comb_dict, x_lims=x_lims, newfig=newfig)
            y_off = -np.linspace(-0.3, 0.3, len(indices)+1)
            for i, trace in enumerate(experiment_class.experiments[indices[0]:indices[-1]+1]):
                add_trace_to_plot(np.array(trace['alpha_i']), y_off[i], colors[i], labeler(trace), alpha=1, 
                                  width=width, npi_comb_dict=npi_comb_dict, cm_names=trace['cm_names'] )

            if default_res is not None:
                add_trace_to_plot(default_res, y_off[-1], "k", default_label, alpha=1, 
                                  width=width, npi_comb_dict=npi_comb_dict, cm_names=default_names)

            plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(0.99, 0.99), fontsize=6)
            plt.title(title, fontsize="10")
            plt.savefig(f'figures/appendix/sensitivity/Fig_{experiment_class.exp_info["tag"]}_{j}.pdf', bbox_inches='tight')
    else:    
        setup_plot(experiment_class, npi_comb_dict, x_lims=x_lims, newfig=newfig)
        y_off = -np.linspace(-0.3, 0.3, len(experiment_class.experiments)+1)
        for i, trace in enumerate(experiment_class.experiments):
            
            if experiment_class.exp_info["tag"] == "gatherings_agg":
                if labeler(trace) == "Weaker":
                    add_trace_to_plot(np.array(trace['alpha_i']), y_off[i], colors[i], labeler(trace), alpha=1, 
                                  width=width, npi_comb_dict=grouped_npis_weaker, cm_names=trace['cm_names'])
                else:
                    add_trace_to_plot(np.array(trace['alpha_i']), y_off[i], colors[i], labeler(trace), alpha=1, 
                                  width=width, npi_comb_dict=grouped_npis_none, cm_names=trace['cm_names'])
                
            else:
                add_trace_to_plot(np.array(trace['alpha_i']), y_off[i], colors[i], labeler(trace), alpha=1, 
                                  width=width, npi_comb_dict=npi_comb_dict, cm_names=trace['cm_names'] )

        if default_res is not None:
            add_trace_to_plot(default_res, y_off[-1], "k", default_label, alpha=1, 
                              width=width, npi_comb_dict=npi_comb_dict, cm_names=default_names)

        plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(0.99, 0.99), fontsize=6)
        plt.title(title, fontsize=10)
        plt.savefig(f'figures/appendix/sensitivity/Fig_{experiment_class.exp_info["tag"]}.pdf', bbox_inches='tight')

In [ ]:
len(all_experiments)

In [ ]:
res_dir = 'sensitivity_analysis/default_cdefault/'
all_experiments = get_all_experiments(res_dir)

In [ ]:
school_npis = {
    'All educational\ninstitutions closed': {
        'npis': ['Primary Schools Closed', 'Secondary Schools Closed', 'Universities Away'],
        'type': 'exclude',
        'color': cols[2],
        'main': True
    },
    'Schools closed': {
        'npis': ['Primary Schools Closed', 'Secondary Schools Closed'],
        'type': 'exclude',
        'color': cols[2],
        'main': False,
    },
    'Universities closed': {
        'npis': ['Universities Away'],
        'type': 'exclude',
        'color': cols[2],
        'main': False,
    },
}

In [ ]:
experiment_classes = make_all_experiment_classes(all_experiments)

for experiment_class in experiment_classes:
    if experiment_class.exp_info['tag'] == "delay_schools":
        plt.figure(figsize=(4, 2), dpi=400)
        plot_experiment_class(experiment_class, school_npis, (-10, 50), default_alpha, default_names, width=2, newfig=False)
    else:
        plot_experiment_class(experiment_class, grouped_npis, (-25, 100), default_alpha, default_names)